In [18]:
import json
import pymongo
import tweepy
import time
from pymongo import MongoClient
import re
import nltk

from nltk.stem import WordNetLemmatizer
import  requests
from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
from textblob import TextBlob

consumer_key = 'MyBZxnITzGZDLDP9x638O94i2'
consumer_secret = 'qDeUuVUIFyJD3PTAdfM7gLyzGV64qnX1CwhVLNTZTAXTAFxJ8v'
access_key = '707833865306411008-sXeJvEZZNEkx9gOXKbSg63RAxJ2BvcZ'
access_secret = 'WnSPq58mgXK9KA95ZcWqX8uCDWCgmX5WYsICA6UxzSZon'

def tweepy1():
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    client = MongoClient()
    db = client.tweetsdb_2
    
    collection = db['rawtweet']
    if collection.count() != 0:
        print('Drop rawtweet collection')
        db.rawtweet.drop() 
    
    class TSListener(tweepy.StreamListener):
        def on_status(self, status):
            try:
                db.rawtweets.insert_one({'text': status.text})  # Here we assign where to save the information
                print(status.text)  # We print the code to see the progress, this is optional
                print('')  # For each tweet printed we print a blank line to read the information more easily 
            except Exception as e:
                print('Encountered Exception:', e, file=sys.stderr)
                pass

        def on_error(self, status_code):
            print(status_code)
            return True # Don't kill the stream
        
        def on_timeout(self):
            print('Timeout...', file=sys.stderr)
            return True # Don't kill the stream
        
    term = input('Keyword: \n')
    print('Gathering tweets....Please wait a moment ....')
    time.sleep(5)
    sapi = tweepy.Stream(auth, TSListener(api))
    sapi.filter(track=[term], languages = ["en"])
    client.close()

def facebook():
    client = MongoClient()
    db = client.tweetsdb_3
    collection = db['rawtweet']
    if collection.count() != 0:
        print('Drop rawtweet collection')
        db.rawtweet.drop()
    token = 'EAACEdEose0cBAL8rw1oE9NOrYdsAnbPi2AkZAZBxjkWRr7r4YIpk4B2TPwX9f1ZBpDvpVmvy7yLKATc7nwtFYHaXhj9gyYjSuoNuk1r9OsG45c34DsQTpAp2sxyxjjZCFbSvcpveJk8uFE9nNEfEqYDt5jJDsd0U0qdZC2i4HXgZDZD'
    res = requests.get('https://graph.facebook.com/v2.3/280034132055819/feed?access_token=%s'%(token))
    jsondata = json.loads(res.text)
    for athletes in jsondata['data']:
        try: db.rawtweets.insert_one({'text': athletes['message']})
        except: print()

def cleanCode():
    wnl = WordNetLemmatizer()
    from nltk.corpus import stopwords
    from nltk.corpus import wordnet as wn
    neg_emo = ['>:[', ':-(', ':(', ':-c', ':c', ':-<\n', ':?C', ':<', ':-[', ':[', ':{',
           ':-||', ':@', ":'-(", ":'(", 'D:<', 'D:', 'D8', 'D;', 'D=', 'DX', 'v.v', 
           "D-':", '>:O', ':-O', ':O', '∞o∞', '∞O∞', ':O', 'o_O', 'o_0', 'o.O', '8-0',
           '>:\\', '>:/', ':-/', ':-.', ':/', ':\\', '=/', '=\\', ':L', '=L', ':S', '>.<', 
           ':-|', ':$', ':-X', ':X', ':-#', ':#', '>:)', '>;)', '>:-)', '}:-)', '}:)', 
           '3:-)', '3:)', ':-&', ':&', '#-)', '%-)', '%)', ':-###..', ':###..', '<:-|', 
           '</3', ':-\\\\', ':\\\\', ':-/', ':/', ')-:', '):', ';-(', ';(', '😩','🙏','😂','😭']
    pos_emo =[':-)', ':)', ':o)', ':]', ':3', ':c)',':>','=]','8)','=)',':}',':^)',':?)',':-D',
          ':D','8-D','8D','x-D','xD','X-D','XD','=-D','=D','=-3','=3','B^D',":-))",'(-:',
          '(:','B-)',';-)',';)',':-P',':P','<3',":'-)",":')",':*',':^*',"('}{' )",'*-)','*)',
          ';-]',';]',';D',';^)',':-,','>:P','X-P','x-p','xp','XP',':-p',':p','=p',':-\xde',
          ':\xde',':-b',':b','O:-)','0:-3','0:3 ','0:-)','0:)','0;^)','o/\\o','^5 ','>_>^ ^<_<',
          '|;-)','|-O','👌','😘','💋','😍']
    totalEmo = pos_emo + neg_emo
    totalEmo2 = set(totalEmo)
    punc = (['!', '#', '"', '%', '$', "'", '&', ')', '(', '+', '*', '-', ',', '/', '.', ';', ':', 
         '=', '<', '?', '>', '[', ']', '\\', '_', '^', '`', '{', '}','💥','🔥','🌷','😉','😌', '|', '~', '↺','🌲','🏁','🙋','🍷','👩','🍸','🍹','⏬','⏬','⏬','⏩','📹'])
    exclude = set(punc)
    stopwords = stopwords.words('english')
    ignore_regex = ["https?[\w:/.?=]*",   # Remove URLs
                    "@\w*",  # Remove @tag
                    # "\\u\w{4}"  # Remove Unicode
                   ]
    def remove(text):
        for i in ignore_regex: 
            p = re.compile(i)
            t = p.sub('', text)
            text = t
        return text
    client = MongoClient()
    db = client.tweetsdb_2
    tweets = db.rawtweets
    db.modtweet.drop()
    
    def is_noun(tag):
        return tag in ['NN', 'NNS', 'NNP', 'NNPS']
    def is_verb(tag):
        return tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    def is_adverb(tag):
        return tag in ['RB', 'RBR', 'RBS']
    def is_adjective(tag):
        return tag in ['JJ', 'JJR', 'JJS']
    def penn_to_wn(tag):
        if is_adjective(tag):
            return wn.ADJ
        elif is_noun(tag):
            return wn.NOUN
        elif is_adverb(tag):
            return wn.ADV
        elif is_verb(tag):
            return wn.VERB
        return None
    
    tw = tweets.find({},{'text':1,'_id':0})
    for x in tw: # x: dictionary object, e.g., {'text': 'Apple iPhone 6 - 64GB - Silver (AT&amp;T)}
        y = x['text'].lower()
        result = y.replace('rt @','@')
        result = result.replace(' rt ','')
        result = result.replace(' rt','')
        st = remove(result) # URL & @user removed
        
        emoticonExt = ' '.join(ch for ch in totalEmo2 if ch in st) 
        st = ''.join(ch for ch in st if ch not in exclude)
        st = ''.join(ch for ch in st if ch not in totalEmo2)
        
        toke = nltk.word_tokenize(st)
        posTag = nltk.pos_tag(toke)
        
        lema_string = ""
        for (x,tag) in posTag:
            if penn_to_wn(tag) == None:
                lema_string = lema_string + ' ' +  wnl.lemmatize((x))                
            else:
                lema_string = lema_string + ' ' +  wnl.lemmatize(x,penn_to_wn(tag))
        
        lema_string = lema_string +' '+ emoticonExt
        emoticonExt = "" 
        swSplit = lema_string.split(' ')
        noSW = [x for x in swSplit if x not in stopwords]
        lema_string = ' '.join(noSW)
        
        print('')
        print('modified: ')
        print(lema_string)
        
        # saving data to a new Collection called "modtweet"
        db.modtweet.insert_one({"text":lema_string})
        print('Cleaning done')

def classifier():
    neg_emo = ['>:[', ':-(', ':(', ':-c', ':c', ':-<\n', ':?C', ':<', ':-[', ':[', ':{', ':-||', ':@', ":'-(", ":'(", 'D:<', 'D:', 'D8', 'D;', 'D=', 'DX', 'v.v', "D-':", '>:O', ':-O', ':O', '∞o∞', '∞O∞', ':O', 'o_O', 'o_0', 'o.O', '8-0', '>:\\', '>:/', ':-/', ':-.', ':/', ':\\', '=/', '=\\', ':L', '=L', ':S', '>.<', ':-|', ':$', ':-X', ':X', ':-#', ':#', '>:)', '>;)', '>:-)', '}:-)', '}:)', '3:-)', '3:)', ':-&', ':&', '#-)', '%-)', '%)', ':-###..', ':###..', '<:-|', '</3', ':-\\\\', ':\\\\', ':-/', ':/', ')-:', '):', ';-(', ';(']
    pos_emo = [':-)', ':)', ':o)', ':]', ':3', ':c)',':>','=]','8)','=)',':}',':^)',':?)',':-D',':D','8-D','8D','x-D','xD','X-D','XD','=-D','=D','=-3','=3','B^D',":-))",'(-:','(:','B-)',';-)',';)',':-P',':P','<3',":'-)",":')",':*',':^*',"('}{' )",'*-)','*)',';-]',';]',';D',';^)',':-,','>:P','X-P','x-p','xp','XP',':-p',':p','=p',':-\xde',':\xde',':-b',':b','O:-)','0:-3','0:3 ','0:-)','0:)','0;^)','o/\\o','^5 ','>_>^ ^<_<','|;-)','|-O']
    totalEmo = pos_emo + neg_emo
    totalEmo2 = set(totalEmo)
    punc = (['{', '}', '"'])
    exclude = set(punc)
    client = MongoClient()
    db = client.tweetsdb_2
    tweets = db.modtweet
    db.pos_tweet.drop()
    db.neg_tweet.drop()
    db.neu_tweet.drop()
    
    with open('word/pos.txt', 'r') as f:
        positive_words = [line.strip() for line in f]
        positive_words = [x.lower() for x in positive_words]
    with open('word/neg.txt', 'r') as f:
        negative_words = [line.strip() for line in f]
        negative_words = [x.lower() for x in negative_words]
    with open('word/pos1.txt', 'r') as f:
        positive_words1 = [line.strip() for line in f]
    with open('word/neg1.txt', 'r') as f:
        negative_words1 = [line.strip() for line in f]
    
    total_positive = positive_words + positive_words1
    total_negative = negative_words + negative_words1
    tw = tweets.find({},{'text':1,'_id':0})
    print('Working...')
    for x in tw:
        tw_string = x['text']
        emoticonExt = ' '.join(ch for ch in totalEmo2 if ch in tw_string)
        json_string = ''.join(ch for ch in tw_string if ch not in exclude)
        json_string = json_string +' '+ emoticonExt
        emoticonExt = ""
        raw_words = json_string.split(" ")
        positive_score1 = len([word for word in raw_words if word in pos_emo])
        negative_score1 = len([word for word in raw_words if word in neg_emo])
        total_score1 = positive_score1 - negative_score1
        
        if (total_score1 > 0):
            #saving data to Positive Collection
            db.pos_tweet.insert_one({"text":json_string})
        elif (total_score1 < 0):
            #saving data to Negative Collection
            db.neg_tweet.insert_one({"text":json_string})
        else:
            positive_score2 = len([word for word in raw_words if word in total_positive])
            negative_score2 = len([word for word in raw_words if word in total_negative])
            total_score2 = positive_score2 - negative_score2
            
            if (total_score2 > 0):
                #saving data to Positive Collection
                db.pos_tweet.insert_one({"text":json_string})
            elif (total_score2 < 0):
                #saving data to Negative Collection
                db.neg_tweet.insert_one({"text":json_string})
            else: # go to next classifier
                tb = TextBlob(json_string)    
                pol = tb.sentiment.polarity
                if (pol > 0):
                    #saving data to Positive Collection
                    db.pos_tweet.insert_one({"text":json_string})
                elif (pol < 0):
                    #saving data to Negative Collection
                    db.neg_tweet.insert_one({"text":json_string})
                else: 
                    db.neu_tweet.insert_one({"text":json_string})
        print('Done.')
        
def mongo2wordcloud():
    client = MongoClient()
    db = client.tweetsdb_2
    mod_tweet = db.neg_tweet.find()
    
    content = []
    for pt in mod_tweet:
        tmp = pt['text'].encode('ascii', 'ignore').decode("utf-8")
        tmp = tmp.replace('\n', ' ')
        content.append(tmp + '\n')
        
    text = open('e:\\PythonFiles\\positive1.txt','w')
    for tweet in content:
        text.write(tweet)
    text.close()
    
    f = open('e:\\PythonFiles\\positive1.txt', 'r')
    text = f.read()
    f.close()
    
    wordcloud = WordCloud(
        font_path='/Users/ada/font/intrastatehighway.ttf',
        stopwords=STOPWORDS,
        background_color='#222222',
        width=1000,height=800).generate(text)
    
    plt.figure(figsize=(12,12))  # figsize = (w, h) w,h tuple in inches
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

#tweepy1()
facebook()
cleanCode()
classifier()
mongo2wordcloud()




modified: 
 mira whatsapp aaaaaaaa 
Cleaning done

modified: 
 whatsapp developer offer endtoend encryption nokia phone however user see new display 
Cleaning done

modified: 
 size 2634 available pick updelivery whatsapp2348091222253 bbm59240cf9 polo 3500 and… 
Cleaning done

modified: 
 whatsapp number choose perfect profile pic initiate chat … 
Cleaning done

modified: 
 accord police juma gun 0930pm whatsapp 25426 active 2205pm rip jj … 
Cleaning done

modified: 
 whatsapp number choose perfect profile pic initiate chat … 
Cleaning done

modified: 
 open member 👉 
Cleaning done

modified: 
 help hi pre order bts young forever daynight ver due 19th may 12 noon rm92wmrm97em whatsapp 0197748160 tq 
Cleaning done

modified: 
 whatsapp developer offer endtoend encryption nokia phone neurogadget 
Cleaning done

modified: 
 open member amp ps info whatsapp 087889355679 
Cleaning done

modified: 
 create meme following message hate people never chat steal whatsapp profile picture 
Cleani

KeyboardInterrupt: 